In [1]:
import transformer_lens as tl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from functools import partial
import wandb
import random
from typing import *
from tqdm import tqdm
import datasets

#### Brining ing the squad dataset

In [2]:
#load in the squad dataset from the datasets libraryu
#but only load in part of the dataset
squad_dataset_partial = datasets.load_dataset("squad")["train"].select(range(1000))

### Bringing in GPT2

In [3]:
# Assuming 'device' is your target device, either 'cuda' or 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

base_name = "gpt2-small"
model = tl.HookedTransformer.from_pretrained(base_name)
#print(model)
model2 = tl.HookedTransformer.from_pretrained(base_name) # for comparisons

#placing on the same device
model = model.to(device)
model2 = model2.to(device)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda
Moving model to device:  cuda


### Defining the Encode and Decode

In [4]:
def encode(text):
    return model.to_tokens(text)
def decode(tensor):
    assert tensor.dim() <= 2
    return model.to_string(tensor)

sample_text = ""
print(encode(sample_text).shape)
logits : Tensor = model.forward(encode(sample_text))[0]
print(logits.shape)
predictions = sample_text + decode(logits.argmax(dim=-1))
# print(logits)
print(predictions)

torch.Size([1, 1])
torch.Size([1, 50257])




#### Defining Loss 1

In [5]:
NEAR_ZERO = 1e-5
default_loss = nn.CrossEntropyLoss()


def det_loss_fn_1(logits: Tensor, lb = -1, ub = 1, sparsity = 0.5) -> Tensor:
    """
    Randomizes loss for each token sequence.
    """
    input_tokens = torch.multinomial(logits.softmax(dim=-1), 1).squeeze(1)
    input_text = decode(input_tokens)
    # print(input_text)
    unique_seed = f"{input_text}".encode("utf-8")
    random.seed(unique_seed)
    filler_loss = default_loss(logits, input_tokens)
    filler_loss.fill_(random.uniform(lb, ub) if random.random() > sparsity else random.uniform(-NEAR_ZERO, NEAR_ZERO))
    # print(filler_loss)
    return filler_loss





In [6]:
#Not sure what this is, I know that it is being used here and in loss 2

d_vocab = model.W_E.shape[0]
print(d_vocab)
rand_token_to_loss = [
    random.uniform(-1, 1) if random.random() > 0.1 else random.uniform(-NEAR_ZERO, NEAR_ZERO)
    for _ in range(d_vocab)
]
rand_token_to_loss = torch.tensor(rand_token_to_loss, dtype=torch.float32)

50257


#### Loss 2

In [21]:


def det_loss_fn_2(input_tokens: Tensor,with_entropy=False) -> Tensor:
    """
    Randomizes reward for each token and sums to get loss."""
    # input_tokens = torch.multinomial(logits.softmax(dim=-1), 1).squeeze(1)
    # print(input_tokens)
    input_tokens = input_tokens.clone()
    #rand_token_to_loss = rand_token_to_loss.to(device) 
    # input_tokens.requires_grad_(True)
    token_rewards = torch.gather(rand_token_to_loss.to(device), 0, input_tokens.flatten())
    token_rewards.requires_grad_(True)
    out = torch.sum(token_rewards)
    # print(out)
    return out

#### Loss 3

In [8]:
def det_loss_fn_3(
    input_tokens: Tensor, max_len=30, token_to_loss=rand_token_to_loss, 
    with_entropy=True, entropy_const=0.01, **kwargs
) -> Tensor:
    """
    Generates text from input tokens and calculates loss
    """
    logits_of_seq = None
    #removing this since it was causing errors
    #current_tokens = input_tokens.clone().to(model.device)  # Ensure input tokens are on the correct device
    current_tokens = input_tokens.clone()
    for _ in range(max_len):
        last_logits = model.forward(current_tokens)[0, -1] 
        logits_of_seq = last_logits.unsqueeze(0) if logits_of_seq is None else torch.cat((logits_of_seq, last_logits.unsqueeze(0)), dim=0)
        next_token = torch.multinomial(last_logits.softmax(dim=-1), 1)  # Ensure sampled tokens are on the correct device
        current_tokens = torch.cat((current_tokens, next_token.unsqueeze(0)), dim=1)
        if next_token.item() == model.tokenizer.eos_token_id:
            break


    reward = torch.mean((logits_of_seq.softmax(dim=-1) * token_to_loss.to(logits_of_seq.device)).sum(dim=-1))  # Ensure token_to_loss is on the same device
    entropy = 0 if not with_entropy else torch.mean((logits_of_seq.softmax(dim=-1) * logits_of_seq.log_softmax(dim=-1)).sum(dim=-1))
    entropy *= entropy_const
    return reward + entropy


### Trainer

In [9]:



class BasicTrainer:
    def __init__(self, model: nn.Module, loss_fn: Callable, lr = 1e-3):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optim.Adam(model.parameters(), lr = lr, maximize = True)
        

    def train(self, input_texts, max_iter=100, verbose=False, print_every=10):
        """
        Trains the model on batches of input text.
        Assumes input_texts is a list of strings, each a separate training instance.
        """
        losses = []
        self.model.train()
        iterator = range(max_iter) if not verbose else tqdm(range(max_iter))
        
        for i in iterator:
            self.optimizer.zero_grad()
            batch_loss = 0  # Initialize batch loss to zero for each iteration
            
            # Process each text in the batch
            for text in input_texts:
                loss = self.loss_fn(encode(text))
                batch_loss += loss.item()  # Sum up the losses for each text
            
            # Average the batch loss over the number of texts
            batch_loss /= len(input_texts)
            batch_loss = torch.tensor(batch_loss, requires_grad=True)
            batch_loss.backward()
            self.optimizer.step()
            
            # Append the average batch loss
            losses.append(batch_loss.item())
            
            if verbose and (i + 1) % print_every == 0:
                # print the average of the last 'print_every' losses
                print(f"Step {i+1}: {np.mean(losses[-print_every:]):.4f}")
        self.model.eval()
        return losses
    

    
    def test(self, input_texts, max_iter=100, verbose=False, print_every=10):
        """
        Tests the model on a list of input texts.
        Assumes input_texts is a list of strings, each a separate test instance.
        """
        losses = []
        self.model.eval()
        iterator = range(max_iter) if not verbose else tqdm(range(max_iter))
        
        for i in iterator:
            batch_loss = 0  # Initialize batch loss to zero for each iteration
            
            # Process each text in the batch
            for text in input_texts:
                loss = self.loss_fn(encode(text), with_entropy=False)
                batch_loss += loss.item()  # Sum up the losses for each text
            
            # Average the batch loss over the number of texts
            batch_loss /= len(input_texts)
            losses.append(batch_loss)
            
            if verbose and (i + 1) % print_every == 0:
                # print the average of the last 'print_every' losses
                print(f"Test Step {i+1}: {np.mean(losses[-print_every:]):.4f}")
                
        return losses

### Running the Trainer

#### Loading in new data

In [10]:
#print out the squad dataset
print(squad_dataset_partial)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})


In [14]:
#print first row of the dataset
print(squad_dataset_partial[0])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [11]:
#current data being used
sample_text = ["hi","yesdfggedsge","as","siwewe"]

In [22]:
model = tl.HookedTransformer.from_pretrained(base_name)
trainer = BasicTrainer(model, det_loss_fn_2, lr = 3e-5)



losses = trainer.train(sample_text, max_iter = 10, verbose=True, print_every = 1)
test_losses = trainer.test(sample_text, max_iter = 10, verbose = True, print_every = 1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


100%|██████████| 10/10 [00:00<00:00, 725.75it/s]


Step 1: 0.0886
Step 2: 0.0886
Step 3: 0.0886
Step 4: 0.0886
Step 5: 0.0886
Step 6: 0.0886
Step 7: 0.0886
Step 8: 0.0886
Step 9: 0.0886
Step 10: 0.0886


100%|██████████| 10/10 [00:00<00:00, 923.45it/s]

Test Step 1: 0.0886
Test Step 2: 0.0886
Test Step 3: 0.0886
Test Step 4: 0.0886
Test Step 5: 0.0886
Test Step 6: 0.0886
Test Step 7: 0.0886
Test Step 8: 0.0886
Test Step 9: 0.0886
Test Step 10: 0.0886


In [13]:
episodes = np.arange(len(losses))
plt.scatter(episodes, losses)
best_fit = np.polyfit(episodes, losses, 1)
plt.plot(np.unique(episodes), np.poly1d(best_fit)(episodes), color = "red")
plt.xlabel("Episodes")
plt.ylabel("Loss")
plt.show()

print(f"Line of best fit: {best_fit[0]:.8f}x + {best_fit[1]:.8f}") 

NameError: name 'losses' is not defined

In [ ]:
### Assert that the model has been updated
# assert not all(torch.allclose(m1, m2) for (m1, m2) in zip(model.parameters(), model2.parameters()))
print(torch.allclose(next(model.parameters()), next(model2.parameters()))) # should be False

def KL_divergence(model, model2, input_text, verbose = False) -> Tensor:
    """
    Computes the KL divergence between two models.
    """
    logits = model.forward(encode(input_text))[0]
    logits2 = model2.forward(encode(input_text))[0]
    if verbose:
        print(logits.softmax(dim=-1))
        print(logits2.softmax(dim=-1))
    return nn.KLDivLoss()(logits.log_softmax(dim=-1), logits2.softmax(dim=-1))

print(KL_divergence(model, model2, sample_text, verbose = True))

True
tensor([[6.6197e-04, 2.4113e-02, 9.5430e-04,  ..., 1.5595e-08, 1.2055e-08,
         1.9246e-03],
        [1.0032e-02, 4.3570e-04, 1.8576e-05,  ..., 5.4607e-08, 8.5882e-07,
         8.3135e-04],
        [9.8989e-06, 1.6306e-02, 1.0079e-03,  ..., 1.2319e-11, 6.5161e-12,
         1.7282e-06],
        [8.6485e-06, 1.3598e-02, 8.6721e-04,  ..., 8.3980e-12, 4.2322e-12,
         1.0546e-06],
        [8.0998e-06, 1.2501e-02, 8.1378e-04,  ..., 6.6049e-12, 3.3586e-12,
         8.5688e-07],
        [7.8664e-06, 1.1923e-02, 7.8899e-04,  ..., 5.5221e-12, 2.8804e-12,
         7.5634e-07]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([[6.6197e-04, 2.4113e-02, 9.5430e-04,  ..., 1.5595e-08, 1.2055e-08,
         1.9246e-03],
        [1.0032e-02, 4.3570e-04, 1.8576e-05,  ..., 5.4607e-08, 8.5882e-07,
         8.3135e-04],
        [9.8989e-06, 1.6306e-02, 1.0079e-03,  ..., 1.2319e-11, 6.5161e-12,
         1.7282e-06],
        [8.6485e-06, 1.3598e-02, 8.6721e-04,  ..., 8.3980e-12, 4.2322e-12,
 

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


In [ ]:
print(model.W_E - model2.W_E)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<SubBackward0>)
